# Analyzing Census ACS data for veterans trends
Done in pandas

Alan Hovorka | The Villages Daily Sun

In [2]:
from glob import glob
import os
import pandas as pd
import numpy as np
%matplotlib inline

#import the library called glob. This targets a file folder and grabs all of the files in there.
#With glob, we can quickly create a list of file names that we can parse data for
#glob also supports wildcards so that we don't have to type exact file names. Just specify a general directory and a general file type. 
#When working with small directories, it's OK to generalize with a wildcard. 
#Greater volume of files and larger files require you to be more specific with your globs because it'll take way longer to generate the list.
#pandas, numpy and matplotlib allow us to manipulate the data and visualize it.
#The OS lib is a "module provides a portable way of using operating system dependent functionality."

In [3]:
acs = glob("data/*/*.csv", recursive = True)
year_list = [2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016]
col_list = []
yrs = []

for d in year_list:
    print(d)
for d in acs:
    print(d)

#glob the directory with the csvs into a list
#create an empty dataframe for our eventual merged/concat dataset of census 
#estimates for 12 years
# print(acs[0])
# len(year_list)
# len(acs)

2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
data/acs_1_yr_vet_status_est/vets_2005_acs.csv
data/acs_1_yr_vet_status_est/vets_2006_acs.csv
data/acs_1_yr_vet_status_est/vets_2007_acs.csv
data/acs_1_yr_vet_status_est/vets_2008_acs.csv
data/acs_1_yr_vet_status_est/vets_2009_acs.csv
data/acs_1_yr_vet_status_est/vets_2010_acs.csv
data/acs_1_yr_vet_status_est/vets_2011_acs.csv
data/acs_1_yr_vet_status_est/vets_2012_acs.csv
data/acs_1_yr_vet_status_est/vets_2013_acs.csv
data/acs_1_yr_vet_status_est/vets_2014_acs.csv
data/acs_1_yr_vet_status_est/vets_2015_acs.csv
data/acs_1_yr_vet_status_est/vets_2016_acs.csv


### List of csvs is:
#### Index 0 = 2005 data and Index = -1 is 2016

In [4]:
# print(acs[0])

#### This next section pulls the csv in, drops margin of estimate and percent columns and creates a trimmed csv. 

In [5]:
df = pd.read_csv(acs[0], header=[0,1])
# df.head()
cols = df.filter(regex='\_MOE_').columns
# print(cols)
df.drop(cols, axis=1, inplace=True)
# df.head()
cols2 = df.filter(regex='\ PERCENT').columns
#print(cols2)
df.drop(cols2, axis=1, inplace=True)
# print(df.dtypes)
# df.to_csv('trimmed.csv', index = False)

#### Declare a function to create a list of years that is the length of the csv's states/territories. This list is used to create identifiable years across 12 csv files for eventual time series analysis. 

In [6]:
def yearcolmaker(y):
    yrs = [y]*52
    return yrs

yrs = yearcolmaker(2005)
# df.info()
print(yrs)

[2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005]


In [7]:
#Insert a column for the year list we just created.
df.insert(2, 'year', yrs)

In [8]:
df.head()
#Check to make sure it appended correctly. 

,GEO.id,GEO.id2,year,GEO.display-label,HC01_EST_VC01,HC02_EST_VC01,HC03_EST_VC01,HC01_EST_VC03,HC02_EST_VC03,HC03_EST_VC03,...,HC03_EST_VC43,HC01_EST_VC44,HC02_EST_VC44,HC03_EST_VC44,HC01_EST_VC46,HC02_EST_VC46,HC03_EST_VC46,HC01_EST_VC47,HC02_EST_VC47,HC03_EST_VC47
,Id,Id2,,Geography,Total; Estimate; Civilian population 18 years and over,Veterans; Estimate; Civilian population 18 years and over,Nonveterans; Estimate; Civilian population 18 years and over,Total; Estimate; Civilian population 18 years and over - PERIOD OF SERVICE - Gulf War veterans,Veterans; Estimate; Civilian population 18 years and over - PERIOD OF SERVICE - Gulf War veterans,Nonveterans; Estimate; Civilian population 18 years and over - PERIOD OF SERVICE - Gulf War veterans,...,Nonveterans; Estimate; POVERTY STATUS IN THE PAST 12 MONTHS - Civilian population 18 years and over,Total; Estimate; POVERTY STATUS IN THE PAST 12 MONTHS - Civilian population 18 years and over - Below poverty in the past 12 months,Veterans; Estimate; POVERTY STATUS IN THE PAST 12 MONTHS - Civilian population 18 years and over - Below poverty in the past 12 months,Nonveterans; Estimate; POVERTY STATUS IN THE PAST 12 MONTHS - Civilian population 18 years and over - Below poverty in the past 12 months,Total; Estimate; DISABILITY STATUS - Civilian population 18 years and over,Veterans; Estimate; DISABILITY STATUS - Civilian population 18 years and over,Nonveterans; Estimate; DISABILITY STATUS - Civilian population 18 years and over,Total; Estimate; DISABILITY STATUS - Civilian population 18 years and over - With any disability,Veterans; Estimate; DISABILITY STATUS - Civilian population 18 years and over - With any disability,Nonveterans; Estimate; DISABILITY STATUS - Civilian population 18 years and over - With any disability
0,0400000US01,1,2005,Alabama,3344721,403950,2940771,NaN,20.3,NaN,...,2940771,14.6,7.5,15.6,3344721,403950,2940771,22.6,30.6,21.6
1,0400000US02,2,2005,Alaska,446969,74482,372487,NaN,30.9,NaN,...,372487,10.0,5.9,10.8,446969,74482,372487,17.3,24.8,15.8
2,0400000US04,4,2005,Arizona,4238996,538880,3700116,NaN,16.7,NaN,...,3700116,12.0,6.0,12.9,4238996,538880,3700116,16.0,25.6,14.6
3,0400000US05,5,2005,Arkansas,2023819,259304,1764515,NaN,19.0,NaN,...,1764515,14.7,8.6,15.6,2023819,259304,1764515,24.0,34.1,22.5
4,0400000US06,6,2005,California,25543447,2193336,23350111,NaN,16.3,NaN,...,23350111,11.4,4.8,12.0,25543447,2193336,23350111,14.8,25.4,13.8


In [9]:
df2=df.dropna(axis=1, how='all')
# Drops columns that contain only null values

## Now let's take the above code and make it scalable in a for function that creates a brand new dataframe with all 12 years of Census ACS data in one spot. 

In [10]:
df_list = []
#declare an empty list for our list of dataframes
#declare a for loop the has two variables and two iterable lists. 
#zip allows you to have multiple iterables in the declaration of the for loop.
for f,g in zip(acs,year_list):
    df3 = pd.read_csv(f, header=[0,1])
    #df3 is a variable that holds the current dataframe the loop is iterating through
    #When it reaches the end of the loop, the frame is added to df_list and the loop starts over, overwriting the previous dataframe
    cols = df3.filter(regex='\_MOE_').columns
    #declare a column varaible that collects a list of column names that fit the filter criteria. We drop these columns because they're not useful
    #As I understand it, Pandas can only handle one filter request at a time. You must filter -> drop -> filter -> drop.
    df3.drop(cols, axis=1, inplace=True)
    cols2 = df3.filter(regex='\ PERCENT').columns
    df3.drop(cols2, axis=1, inplace=True)
    cols3 = df3.filter(regex='.Percent;').columns
    df3.drop(cols3, axis=1, inplace=True)
    df3.dropna(axis=1, how='all',inplace=True)
    #call our year list creating function. Here we iterate through a list of years that matches our list of csvs.
    #We want to insert a column that adds a year value to each row
    yrs = yearcolmaker(g)
    df3.insert(2,'year',yrs)
    df3.info()
    df_list.append(df3)
    #Here we add the results of the loop into a list and continue iterating until there are no csv files left
    # https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Columns: 108 entries, (GEO.id, Id) to (HC03_EST_VC47, Nonveterans; Estimate; DISABILITY STATUS - Civilian population 18 years and over - With any disability)
dtypes: float64(79), int64(27), object(2)
memory usage: 44.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Columns: 108 entries, (GEO.id, Id) to (HC03_EST_VC47, Nonveterans; Estimate; DISABILITY STATUS - Civilian population 18 years and over - With any disability)
dtypes: float64(79), int64(27), object(2)
memory usage: 44.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Columns: 109 entries, (GEO.id, Id) to (HC03_EST_VC48, Nonveterans; Estimate; DISABILITY STATUS - Civilian population 18 years and over - With any disability)
dtypes: float64(80), int64(27), object(2)
memory usage: 44.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Columns: 109 entries, (GEO.id, Id) to (HC03_EST_VC4

In [21]:
df_list[6].head()

,GEO.id,GEO.id2,year,GEO.display-label,HC01_EST_VC01,HC02_EST_VC01,HC03_EST_VC01,HC02_EST_VC03,HC02_EST_VC04,HC02_EST_VC05,...,HC03_EST_VC55,HC01_EST_VC56,HC02_EST_VC56,HC03_EST_VC56,HC01_EST_VC60,HC02_EST_VC60,HC03_EST_VC60,HC01_EST_VC61,HC02_EST_VC61,HC03_EST_VC61
,Id,Id2,,Geography,Total; Estimate; Civilian population 18 years and over,Veterans; Estimate; Civilian population 18 years and over,Nonveterans; Estimate; Civilian population 18 years and over,Veterans; Estimate; PERIOD OF SERVICE - Gulf War (9/2001 or later) veterans,Veterans; Estimate; PERIOD OF SERVICE - Gulf War (8/1990 to 8/2001) veterans,Veterans; Estimate; PERIOD OF SERVICE - Vietnam era veterans,...,Nonveterans; Estimate; POVERTY STATUS IN THE PAST 12 MONTHS - Civilian population 18 years and over for whom poverty status is determined,Total; Estimate; POVERTY STATUS IN THE PAST 12 MONTHS - Below poverty in the past 12 months,Veterans; Estimate; POVERTY STATUS IN THE PAST 12 MONTHS - Below poverty in the past 12 months,Nonveterans; Estimate; POVERTY STATUS IN THE PAST 12 MONTHS - Below poverty in the past 12 months,Total; Estimate; DISABILITY STATUS - Civilian population 18 years and over for whom poverty status is determined,Veterans; Estimate; DISABILITY STATUS - Civilian population 18 years and over for whom poverty status is determined,Nonveterans; Estimate; DISABILITY STATUS - Civilian population 18 years and over for whom poverty status is determined,Total; Estimate; DISABILITY STATUS - With any disability,Veterans; Estimate; DISABILITY STATUS - With any disability,Nonveterans; Estimate; DISABILITY STATUS - With any disability
0,0400000US01,1,2011,Alabama,3662910,395753,3267157,14.3,20.4,36.3,...,3172137,16.4,7.9,17.5,3561730,389593,3172137,19.9,29.6,18.7
1,0400000US02,2,2011,Alaska,517799,72407,445392,22.8,29.8,32.1,...,438323,9.2,4.7,9.9,509763,71440,438323,13.4,21.1,12.1
2,0400000US04,4,2011,Arizona,4842927,533608,4309319,13.1,15.4,34.8,...,4200642,16.3,7.6,17.3,4728740,528098,4200642,14.3,25.0,12.9
3,0400000US05,5,2011,Arkansas,2221409,238790,1982619,11.8,15.7,35.6,...,1918980,16.7,8.3,17.7,2152265,233285,1918980,20.7,32.7,19.3
4,0400000US06,6,2011,California,28292703,1910994,26381709,11.6,14.6,34.6,...,25861216,14.5,7.1,15.1,27739741,1878525,25861216,12.5,25.4,11.5


In [12]:
print(year_list[6])

2011


#### We've successfully created a list of cleaned dataframes. Each dataframes has a column to identify which year it is. Now we need to merge the data. In theory we could the data at the start and then filter it. 

In [17]:
df_cc = pd.concat(df_list)

In [18]:
df_cc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 0 to 51
Columns: 556 entries, (GEO.display-label, Geography) to (year, )
dtypes: float64(551), int64(3), object(2)
memory usage: 2.7+ MB


In [20]:
df_cc.to_csv('concated.csv', index = False)

In [22]:
df_list2 = []
for f,g in zip(acs,year_list):
    df3 = pd.read_csv(f, header=[0,1])
    yrs = yearcolmaker(g)
    df3.insert(2,'year',yrs)
    df3.info()
    df_list2.append(df3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Columns: 244 entries, (GEO.id, Id) to (HC03_MOE_VC50, Nonveterans; Margin of Error; PERCENT IMPUTED - Period of service for the civilian veteran population 18 years and over)
dtypes: float64(190), int64(52), object(2)
memory usage: 99.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Columns: 244 entries, (GEO.id, Id) to (HC03_MOE_VC50, Nonveterans; Margin of Error; PERCENT IMPUTED - Period of service for the civilian veteran population 18 years and over)
dtypes: float64(190), int64(52), object(2)
memory usage: 99.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Columns: 250 entries, (GEO.id, Id) to (HC03_MOE_VC51, Nonveterans; Margin of Error; PERCENT IMPUTED - Period of service for the civilian veteran population 18 years and over)
dtypes: float64(196), int64(52), object(2)
memory usage: 101.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to

In [23]:
df_cc2 = pd.concat(df_list2)

In [26]:
df_cc2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 0 to 51
Columns: 1548 entries, (GEO.display-label, Geography) to (year, )
dtypes: float64(1542), int64(4), object(2)
memory usage: 7.4+ MB


In [27]:
df_cc2.to_csv('concated_unfiltered.csv', index = False)

## Concatted data observations
Tables for 2005-2006, near identical structure
Tables for 2007-2014, same number of columns, varying structure
Tables for 2015-2016, near identical structure

Here's how the columns vary: 
    
    2005-2006 = the same
    2007-2010 = the same
    2010-2016 = the same

### Various notes from other people
If the columns have the same naming scheme you could write some code that simplifies a single table, then use a for loop to do that process each table in a for loop before combining them.
 
Basically, you have a list of filenames and years, loop through each filename/year and create a simplified version of the table and put add it to a list then call pd.concat([list of tables]) to bind all the rows together
But, there are a bunch of other ways to do it. It sort of depends on how consistent the data is over time

My thought is you need to loop through each filename, pick out the columns you need, add a column for the year and add it to a list. After that you can smoosh it together with `pd.concat(list)`

#### Pandas snippets

In [13]:
# Some helpful pandas snippets 
# https://gist.github.com/bsweger/e5817488d161f37dcbd2
# Get rid of non-numeric values throughout a DataFrame:
# for col in refunds.columns.values:
#   refunds[col] = refunds[col].replace('[^0-9]+.-', '', regex=True)

In [14]:
# Concatenate two DataFrame columns into a new, single column
# (useful when dealing with composite keys, for example)
# (h/t @makmanalp for improving this one!)
# df['newcol'] = df['col1'].astype(str) + df['col2'].astype(str)

In [15]:
# Create a DataFrame from a Python dictionary
# df = pd.DataFrame(list(a_dictionary.items()), columns = ['column1', 'column2'])